<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT20/blob/master/template-report-lab-X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix Factorization**
**Joakim Eriksson**

# **Abstract**

The objective of this lab was to implement a sparce matrix-vector product, a QR factorization funtion and a direct solver of a matrix vector equation solver.
The algorithms used was taken from the lecture notes,
and when implemented gave small errros in the tests.
This could probably have been solve by using other algorithms.

#**About the code**

In [ ]:
# Copyright (C) 2021 Joakim Eriksson (joakieri@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# **Set up environment**

In [ ]:
# Load neccessary modules.
from google.colab import files

import math
import numpy as np

# **Introduction**

This report tries to solve the problem of implementing functions for computing 
sparce matrix product, QR factorisation of a matrix and a solver function for the equation type $Ax = b$, where $A$ is a matrix and $x$ and $b$ are vectors.
The algorithms used when solving the problems where taken from the lecture notes, and was choosen based on how well I undestood them.

All the matrices used in the tests of the functions were real and quadratic and the compressed row storage(CRS) format was used to represent a sparce matrix. 

# **Method**

I wrote a class to represent CRS matrix,
to easily build a matrix from a list.
The defenitions of the algorithms were taken from the lecture notes and adjusted where nessesary to work in python.

To implement the first function I used algorithm 5.9, sparce_matrix_vector_product, from the lecture notes.
Its implementation in Python was straight forward using the MatrixCRS class,
and not much had to change from the definition in order to get it to work.

To implement the second function I used algorithm 5.3,
modified_gram_schmidt_iteration, from the lecture notes.
This algorithm tooksome time to implement.
There where a lot of operations in this algorithm that I had to implement
because I used regular Python lists instead of Numpy arrays.

To implement the third function I used algorithm 5.2, backward_substitution, from the lecture notes, togeather with my implementation of a QR factorisation algorithm
in order to create a solver function.
The algorithm uses the fact that a matrix can be factorized into a Q and R matrix, where Q is an orthogonal matrix and R is an upper triangular matrix.
This gives you the equation $Ax = b \Leftrightarrow Rx = Q^Tb$.
This lets you solve the equation using backwards sustitution.


In [ ]:
# 1. Function: sparse matrix-vector product
def matrix(r, c, l):
    m = [[0]*c for _ in range(r)]
    for i in range(r):
        for j in range(c):
            m[i][j] = l[i*c + j]
    return m

class MatrixCRS:
    def __init__(self, r, c, l):
        self.val = []
        self.col_idx = []
        self.row_ptr = []
        index = 0
        for i in range(r):
            for j in range(c):
                v = l[j + c*i]
                if v != 0:
                    if index == i:
                        self.row_ptr.append(len(self.val))
                        index += 1
                    self.val.append(v)
                    self.col_idx.append(j)
            if index == i:
                self.row_ptr.append(len(self.val))
        self.row_ptr.append(len(self.val))

    def __str__(self):
        return "val: {}\ncol_idx: {}\nrow_ptr: {}".format(self.val, self.col_idx, self.row_ptr)

def sparce_matrix_vector_product(A, x):
    b = []
    for i in range(len(A.row_ptr) - 1):
        b.append(0)
        for j in range(A.row_ptr[i], A.row_ptr[i+1]):
            b[i] += A.val[j] * x[A.col_idx[j]]
    return b


# 2. Function: QR factorization
def column(A, j):
    v = []
    for a in A:
        v.append(a[j])
    return v

def scalar_product(x, y):
    s = 0
    for i in range(len(x)):
        s += x[i] * y[i]
    return s

def norm(v):
    sum = 0
    for s in v:
        sum += s * s
    return math.sqrt(sum)

def vector_vector_sub(u, v):
    w = [0] * len(u)
    for i in range(len(u)):
        w[i] = u[i] - v[i]
    return w

def scalar_vector_product(s, v):
    u = [0] * len(v)
    for i in range(len(v)):
        u[i] = s * v[i]
    return u

def qr_factorization(A):
    R = [[0] * len(A[0]) for _ in A]
    Q = [[0] * len(A[0]) for _ in A]
    for j in range(len(A)):
        v = column(A,j)
        for i in range(j):
            R[i][j] = scalar_product(column(Q, i), v)
            v = vector_vector_sub(v, scalar_vector_product(R[i][j], column(Q, i)))

        R[j][j] = norm(v)
        v = [x / R[j][j] for x in v]

        for i in range(len(Q)):
            Q[i][j] = v[i]

    return (Q, R)

# 3. Function: direct solver Ax=b

def transpose(A):
    B = [[0]*len(A) for _ in A[0]]
    for i in range(len(A)):
        for j in range(len(A[0])):
            B[j][i] = A[i][j]
    return B

def matrix_vector_product(A, x):
    b = [0] * len(A)
    for i in range(len(A)):
        b[i] = scalar_product(A[i], x)
    return b

def backwards_substitution(U, b):
    n = len(U)
    x = [0] * n
    x[n-1] = b[n-1] / U[n-1][n-1]
    for i in range(n-2, -1, -1):
        sum = 0
        for j in range(i+1, n):
            sum += U[i][j] * x[j]
        x[i] = (b[i] - sum) / U[i][i]
    return x

def direct_solver(A, b):
    qr = qr_factorization(A)
    return backwards_substitution(qr[1], matrix_vector_product(transpose(qr[0]), b))

# Test 1
l = [3,2,0,2,0,0,0,2,1,0,0,0,0,0,1,0,0,0,0,0,3,2,0,0,0,0,0,0,1,0,0,0,0,0,2,3]
x = [1,2,3,4,5,6]
M = MatrixCRS(6, 6, l)
N = np.array(matrix(6, 6, l))

R1 = np.array(sparce_matrix_vector_product(M, x))
R2 = N.dot(x)
assert((R1 == R2).all())

#Test 2
l = [1,1,0,1,0,1,0,1,1]
A = np.array(matrix(3, 3, l))
I = np.identity(3)
qr = qr_factorization(A.tolist())
Q = np.array(qr[0])
R = np.array(qr[1])

assert(np.allclose(R, np.triu(R)))
#print(np.linalg.norm(np.matmul(Q.T, Q) - np.identity(3), 'fro'))
assert(np.linalg.norm(np.matmul(Q.T, Q) - np.identity(3), 'fro') == 0)
#print(np.linalg.norm(np.matmul(Q ,R) - np.array(A)))
assert(np.linalg.norm(np.matmul(Q ,R) - np.array(A)) == 0)

#Test 3
y = np.array([1,2,3])
b = A.dot(y)
x = np.array(direct_solver(A.tolist(), b.tolist()))

#print(np.linalg.norm(np.matmul(A, x) - b))
assert(np.linalg.norm(np.matmul(A, x) - b) == 0)
#print(np.linalg.norm(x - y))
assert(np.linalg.norm(x - y) == 0)

# **Results**

The code passes the first test, but the other two fails.
In the second test case, the algorithm manages to factorize the input matrix into an orthogonal matrix and upper triangular matrix.

The erros from the second and thrid tests were:
$$||Q^TQ-I||_F = 4.0542616684110986 \times 10^{-16},$$
$$||QR-A||_F = 1.6208456643253248 \times 10^{-16},$$
$$||Ax-b||_F = 1.2560739669470201 \times 10^{-15},$$
$$||x-y||_F = 1.1801832636420706 \times 10^{-15}.$$

# **Discussion**

The results were not surprising.
Using an other, more accurate, algorithm for the second function might have given a better result.
Since the thrid algorithm relies on the second one it is not a surprice that it also fails.
The implication of this error is that it will probably become larger when larger matrices and vectors are used, as the matrices and vectors used in the test case is quite small.

Looking at the number of function used to implement the second algorithm, I will concider using Numpy to represent vectors and matrices instead of list, as these implementation already have the basic operations for vectors and matrices implemented and are easier to use.
